In [1]:
#import libraries
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
file_path= 'data/labelled_imdb.csv'
movies = pd.read_csv(file_path)
movies.head()

,Unnamed: 0.1,Unnamed: 0,movie,genre,summary,labelled_genre
0,0,0,Bullet Train,"Action,Comedy,Thriller",Five assassins aboard a swiftly-moving bullet ...,Action
1,1,1,Black Adam,"Action,Adventure,Fantasy","Nearly 5,000 years after he was bestowed with ...",Action
2,2,2,Top Gun: Maverick,"Action,Drama","After thirty years, Maverick is still pushing ...",Action
3,3,3,The Woman King,"Action,Drama,History",A historical epic inspired by true events that...,Action
4,4,4,Thor: Love and Thunder,"Action,Adventure,Comedy","Thor enlists the help of Valkyrie, Korg and ex...",Action


In [5]:
unnamed_cols  =  movies.columns.str.contains('Unnamed')
movies = movies.drop(movies[movies.columns[unnamed_cols]], axis=1)

movies.head()

,movie,genre,summary,labelled_genre
0,Bullet Train,"Action,Comedy,Thriller",Five assassins aboard a swiftly-moving bullet ...,Action
1,Black Adam,"Action,Adventure,Fantasy","Nearly 5,000 years after he was bestowed with ...",Action
2,Top Gun: Maverick,"Action,Drama","After thirty years, Maverick is still pushing ...",Action
3,The Woman King,"Action,Drama,History",A historical epic inspired by true events that...,Action
4,Thor: Love and Thunder,"Action,Adventure,Comedy","Thor enlists the help of Valkyrie, Korg and ex...",Action


In [6]:
movies['first_genre'] = movies['genre'].str.split(',')
movies['first_genre'] = movies['first_genre'].apply(lambda x: x[0])
movies

,movie,genre,summary,labelled_genre,first_genre
0,Bullet Train,"Action,Comedy,Thriller",Five assassins aboard a swiftly-moving bullet ...,Action,Action
1,Black Adam,"Action,Adventure,Fantasy","Nearly 5,000 years after he was bestowed with ...",Action,Action
2,Top Gun: Maverick,"Action,Drama","After thirty years, Maverick is still pushing ...",Action,Action
3,The Woman King,"Action,Drama,History",A historical epic inspired by true events that...,Action,Action
4,Thor: Love and Thunder,"Action,Adventure,Comedy","Thor enlists the help of Valkyrie, Korg and ex...",Action,Action
...,...,...,...,...,...
62614,Underland Episode III: Beyond the Rift,"Action,Adventure,Comedy",UNDERLAND' takes us along for the journey of a...,Sci-Fi,Action
62615,The Turtle and the Time Machine,"Comedy,Drama,Sci-Fi",Three 20-somethings roam the Tennessee country...,Sci-Fi,Comedy
62616,Form Function,"Documentary,Short,Sci-Fi",This purely visual short film shows an industr...,Sci-Fi,Documentary
62617,Forge,"Drama,Family,Sci-Fi",A man tries to save his reclusive scientist br...,Sci-Fi,Drama


In [7]:
movies=movies.drop_duplicates(subset='movie')
movies

,movie,genre,summary,labelled_genre,first_genre
0,Bullet Train,"Action,Comedy,Thriller",Five assassins aboard a swiftly-moving bullet ...,Action,Action
1,Black Adam,"Action,Adventure,Fantasy","Nearly 5,000 years after he was bestowed with ...",Action,Action
2,Top Gun: Maverick,"Action,Drama","After thirty years, Maverick is still pushing ...",Action,Action
3,The Woman King,"Action,Drama,History",A historical epic inspired by true events that...,Action,Action
4,Thor: Love and Thunder,"Action,Adventure,Comedy","Thor enlists the help of Valkyrie, Korg and ex...",Action,Action
...,...,...,...,...,...
62613,The EBay Pickup,"Documentary,Short,Action","After winning a bid on EBay, Pat travels far f...",Sci-Fi,Documentary
62614,Underland Episode III: Beyond the Rift,"Action,Adventure,Comedy",UNDERLAND' takes us along for the journey of a...,Sci-Fi,Action
62615,The Turtle and the Time Machine,"Comedy,Drama,Sci-Fi",Three 20-somethings roam the Tennessee country...,Sci-Fi,Comedy
62616,Form Function,"Documentary,Short,Sci-Fi",This purely visual short film shows an industr...,Sci-Fi,Documentary


In [11]:
movies = movies[movies["summary"].str.contains("Add a Plot") == False]
movies.reset_index(drop=True)

,movie,genre,summary,labelled_genre,first_genre
0,Bullet Train,"Action,Comedy,Thriller",Five assassins aboard a swiftly-moving bullet ...,Action,Action
1,Black Adam,"Action,Adventure,Fantasy","Nearly 5,000 years after he was bestowed with ...",Action,Action
2,Top Gun: Maverick,"Action,Drama","After thirty years, Maverick is still pushing ...",Action,Action
3,The Woman King,"Action,Drama,History",A historical epic inspired by true events that...,Action,Action
4,Thor: Love and Thunder,"Action,Adventure,Comedy","Thor enlists the help of Valkyrie, Korg and ex...",Action,Action
...,...,...,...,...,...
40170,The EBay Pickup,"Documentary,Short,Action","After winning a bid on EBay, Pat travels far f...",Sci-Fi,Documentary
40171,Underland Episode III: Beyond the Rift,"Action,Adventure,Comedy",UNDERLAND' takes us along for the journey of a...,Sci-Fi,Action
40172,The Turtle and the Time Machine,"Comedy,Drama,Sci-Fi",Three 20-somethings roam the Tennessee country...,Sci-Fi,Comedy
40173,Form Function,"Documentary,Short,Sci-Fi",This purely visual short film shows an industr...,Sci-Fi,Documentary


In [12]:
def clean_dataset(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text

In [13]:
movies['clean_summary'] = movies['summary'].apply(lambda x: clean_dataset(x))
movies.head(20)

,movie,genre,summary,labelled_genre,first_genre,clean_summary
0,Bullet Train,"Action,Comedy,Thriller",Five assassins aboard a swiftly-moving bullet ...,Action,Action,five assassins aboard a swiftly moving bullet ...
1,Black Adam,"Action,Adventure,Fantasy","Nearly 5,000 years after he was bestowed with ...",Action,Action,nearly years after he was bestowed with the al...
2,Top Gun: Maverick,"Action,Drama","After thirty years, Maverick is still pushing ...",Action,Action,after thirty years maverick is still pushing t...
3,The Woman King,"Action,Drama,History",A historical epic inspired by true events that...,Action,Action,a historical epic inspired by true events that...
4,Thor: Love and Thunder,"Action,Adventure,Comedy","Thor enlists the help of Valkyrie, Korg and ex...",Action,Action,thor enlists the help of valkyrie korg and ex ...
5,Black Panther: Wakanda Forever,"Action,Adventure,Drama",The nation of Wakanda is pitted against interv...,Action,Action,the nation of wakanda is pitted against interv...
6,Lou,"Action,Crime,Drama",A storm rages. A young girl is kidnapped. Her ...,Action,Action,a storm rages a young girl is kidnapped her mo...
7,Everything Everywhere All at Once,"Action,Adventure,Comedy",An aging Chinese immigrant is swept up in an i...,Action,Action,an aging chinese immigrant is swept up in an i...
8,Avatar: The Way of Water,"Action,Adventure,Fantasy",Jake Sully lives with his newfound family form...,Action,Action,jake sully lives with his newfound family form...
9,Last Seen Alive,"Action,Drama,Mystery",Will's soon-to-be ex-wife mysteriously vanishe...,Action,Action,wills soon to be ex wife mysteriously vanishes...


In [14]:
del movies['summary']
movies.head(20)

,movie,genre,labelled_genre,first_genre,clean_summary
0,Bullet Train,"Action,Comedy,Thriller",Action,Action,five assassins aboard a swiftly moving bullet ...
1,Black Adam,"Action,Adventure,Fantasy",Action,Action,nearly years after he was bestowed with the al...
2,Top Gun: Maverick,"Action,Drama",Action,Action,after thirty years maverick is still pushing t...
3,The Woman King,"Action,Drama,History",Action,Action,a historical epic inspired by true events that...
4,Thor: Love and Thunder,"Action,Adventure,Comedy",Action,Action,thor enlists the help of valkyrie korg and ex ...
5,Black Panther: Wakanda Forever,"Action,Adventure,Drama",Action,Action,the nation of wakanda is pitted against interv...
6,Lou,"Action,Crime,Drama",Action,Action,a storm rages a young girl is kidnapped her mo...
7,Everything Everywhere All at Once,"Action,Adventure,Comedy",Action,Action,an aging chinese immigrant is swept up in an i...
8,Avatar: The Way of Water,"Action,Adventure,Fantasy",Action,Action,jake sully lives with his newfound family form...
9,Last Seen Alive,"Action,Drama,Mystery",Action,Action,wills soon to be ex wife mysteriously vanishes...


In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/niharika/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

movies['clean_summary'] = movies['clean_summary'].apply(lambda x: remove_stopwords(x))
movies

,movie,genre,labelled_genre,first_genre,clean_summary
0,Bullet Train,"Action,Comedy,Thriller",Action,Action,five assassins aboard swiftly moving bullet tr...
1,Black Adam,"Action,Adventure,Fantasy",Action,Action,nearly years bestowed almighty powers egyptian...
2,Top Gun: Maverick,"Action,Drama",Action,Action,thirty years maverick still pushing envelope t...
3,The Woman King,"Action,Drama,History",Action,Action,historical epic inspired true events took plac...
4,Thor: Love and Thunder,"Action,Adventure,Comedy",Action,Action,thor enlists help valkyrie korg ex girlfriend ...
...,...,...,...,...,...
62613,The EBay Pickup,"Documentary,Short,Action",Sci-Fi,Documentary,winning bid ebay pat travels far home pick unu...
62614,Underland Episode III: Beyond the Rift,"Action,Adventure,Comedy",Sci-Fi,Action,underland takes us along journey young girl th...
62615,The Turtle and the Time Machine,"Comedy,Drama,Sci-Fi",Sci-Fi,Comedy,three somethings roam tennessee countryside se...
62616,Form Function,"Documentary,Short,Sci-Fi",Sci-Fi,Documentary,purely visual short film shows industrial desi...


In [17]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

movies['stemmed_summary'] = movies['clean_summary'].str.split()
movies['stemmed_summary'] = movies['stemmed_summary'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
movies['stemmed_summary'] = movies['stemmed_summary'].apply(lambda x: ' '.join(x))
movies

,movie,genre,labelled_genre,first_genre,clean_summary,stemmed_summary
0,Bullet Train,"Action,Comedy,Thriller",Action,Action,five assassins aboard swiftly moving bullet tr...,five assassin aboard swift move bullet train f...
1,Black Adam,"Action,Adventure,Fantasy",Action,Action,nearly years bestowed almighty powers egyptian...,near year bestow almighti power egyptian god i...
2,Top Gun: Maverick,"Action,Drama",Action,Action,thirty years maverick still pushing envelope t...,thirti year maverick still push envelop top na...
3,The Woman King,"Action,Drama,History",Action,Action,historical epic inspired true events took plac...,histor epic inspir true event took place kingd...
4,Thor: Love and Thunder,"Action,Adventure,Comedy",Action,Action,thor enlists help valkyrie korg ex girlfriend ...,thor enlist help valkyri korg ex girlfriend ja...
...,...,...,...,...,...,...
62613,The EBay Pickup,"Documentary,Short,Action",Sci-Fi,Documentary,winning bid ebay pat travels far home pick unu...,win bid ebay pat travel far home pick unusu it...
62614,Underland Episode III: Beyond the Rift,"Action,Adventure,Comedy",Sci-Fi,Action,underland takes us along journey young girl th...,underland take us along journey young girl thr...
62615,The Turtle and the Time Machine,"Comedy,Drama,Sci-Fi",Sci-Fi,Comedy,three somethings roam tennessee countryside se...,three someth roam tennesse countrysid search t...
62616,Form Function,"Documentary,Short,Sci-Fi",Sci-Fi,Documentary,purely visual short film shows industrial desi...,pure visual short film show industri design cr...


In [18]:
del movies['clean_summary']
movies

In [19]:
movies.to_csv('data/cleaned_imdb.csv')

,movie,genre,labelled_genre,first_genre,stemmed_summary
0,Bullet Train,"Action,Comedy,Thriller",Action,Action,five assassin aboard swift move bullet train f...
1,Black Adam,"Action,Adventure,Fantasy",Action,Action,near year bestow almighti power egyptian god i...
2,Top Gun: Maverick,"Action,Drama",Action,Action,thirti year maverick still push envelop top na...
3,The Woman King,"Action,Drama,History",Action,Action,histor epic inspir true event took place kingd...
4,Thor: Love and Thunder,"Action,Adventure,Comedy",Action,Action,thor enlist help valkyri korg ex girlfriend ja...
...,...,...,...,...,...
62613,The EBay Pickup,"Documentary,Short,Action",Sci-Fi,Documentary,win bid ebay pat travel far home pick unusu it...
62614,Underland Episode III: Beyond the Rift,"Action,Adventure,Comedy",Sci-Fi,Action,underland take us along journey young girl thr...
62615,The Turtle and the Time Machine,"Comedy,Drama,Sci-Fi",Sci-Fi,Comedy,three someth roam tennesse countrysid search t...
62616,Form Function,"Documentary,Short,Sci-Fi",Sci-Fi,Documentary,pure visual short film show industri design cr...


In [20]:
# Visualize no of movies per genre

# Decide based on visualization if more data of a specific genre is needed
# which column of genre to be used

# Length of summaries

In [21]:
movies.plot.hist(column=['labelled_genre'])

TypeError: no numeric data to plot